In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing data

In [62]:
Train = pd.read_csv('Train.csv')
Riders = pd.read_csv('Riders.csv')
Test = pd.read_csv('Test.csv')

## Train Data cleaning

In [63]:
Riders.drop(['No_Of_Orders','Age','No_of_Ratings'], axis=1, inplace=True)

In [64]:
Train.drop(['Order No','User Id','Vehicle Type','Precipitation in millimeters'], axis=1, inplace=True)

The following columns were deleted because they contained redundant information, from  the data it was observed that most of the deliveries about 98% are same day drliveries which rendered most of the columns redundant so they redundencies were removed.

In [65]:
Train.drop(['Confirmation - Day of Month','Confirmation - Weekday (Mo = 1)','Arrival at Pickup - Day of Month',
            'Arrival at Pickup - Weekday (Mo = 1)','Arrival at Destination - Weekday (Mo = 1)','Pickup - Day of Month',
            'Pickup - Weekday (Mo = 1)','Arrival at Destination - Day of Month','Temperature'], axis=1, inplace=True)

## Transorming coordinates into cities

In [66]:
pickup_co= Train[['Pickup Lat', 'Pickup Long']].apply(tuple, axis=1).tolist()
destination_co = Train[['Destination Lat', 'Destination Long']].apply(tuple, axis=1).tolist()

In [67]:
import reverse_geocoder as rg

pickup_re = rg.search(pickup_co, mode=2)
Train['Pickup_city'] = [x['name'] for x in pickup_re]

dest_re = rg.search(destination_co, mode=2)
Train['Destination_city'] = [x['name'] for x in dest_re]

In [68]:
Train['Pickup_to_Destination']= Train[['Pickup_city', 'Destination_city']].agg('-'.join, axis=1)

In [69]:
Train.drop(['Pickup Lat','Pickup Long','Destination Lat','Destination Long',
            'Pickup_city','Destination_city'], axis=1, inplace=True)

In [70]:
def time_to_seconds(column_name):
    Train[column_name] = pd.to_datetime(Train[column_name])
    return (Train[column_name]- pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')

In [71]:
Train['Confirmation - Time']=time_to_seconds('Confirmation - Time')

In [72]:
Train['Con_Time_of_day'] = ['Morning' if x< 43200 else 'Afternoon' for x in Train['Confirmation - Time']]

In [73]:
Train['Platform Type'] = ['Busy' if x == 3  else 'Not_busy' for x in Train['Platform Type']]

In [74]:
Train.drop(['Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Pickup - Time',
            'Arrival at Destination - Time'], axis=1, inplace=True)

In [75]:
Train.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Rider Id,Time from Pickup to Arrival,Pickup_to_Destination,Con_Time_of_day
0,Busy,Business,9,5,4,Rider_Id_432,745,Nairobi-Nairobi,Morning
1,Busy,Personal,12,5,16,Rider_Id_856,1993,Pumwani-Nairobi,Morning
2,Busy,Business,30,2,3,Rider_Id_155,455,Pumwani-Nairobi,Afternoon
3,Busy,Business,15,5,9,Rider_Id_855,1341,Nairobi-Nairobi,Morning
4,Not_busy,Personal,13,1,9,Rider_Id_770,1214,Nairobi-Nairobi,Morning


In [76]:
Test.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,...,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,...,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,...,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,...,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,...,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


## Test data cleaning

In [77]:
Test.drop(['Order No','User Id','Vehicle Type','Precipitation in millimeters'], axis=1, inplace=True)

In [78]:
Test.drop(['Confirmation - Day of Month','Confirmation - Weekday (Mo = 1)','Arrival at Pickup - Day of Month',
            'Arrival at Pickup - Weekday (Mo = 1)','Pickup - Day of Month',
            'Pickup - Weekday (Mo = 1)','Temperature'], axis=1, inplace=True)

## Coordinate data transformation

In [79]:
pickup_coo= Test[['Pickup Lat', 'Pickup Long']].apply(tuple, axis=1).tolist()
destination_coo = Test[['Destination Lat', 'Destination Long']].apply(tuple, axis=1).tolist()

In [80]:
pickup_ree = rg.search(pickup_coo, mode=2)
Test['Pickup_city'] = [x['name'] for x in pickup_ree]

dest_ree = rg.search(destination_coo, mode=2)
Test['Destination_city'] = [x['name'] for x in dest_ree]

In [81]:
Test['Pickup_to_Destination']= Test[['Pickup_city', 'Destination_city']].agg('-'.join, axis=1)

In [82]:
Test.drop(['Pickup Lat','Pickup Long','Destination Lat','Destination Long',
           'Pickup_city','Destination_city'], axis=1, inplace=True)

In [83]:
def time_to_seconds1(column_name):
    Test[column_name] = pd.to_datetime(Test[column_name])
    return (Test[column_name]- pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')

In [84]:
Test['Confirmation - Time']=time_to_seconds1('Confirmation - Time')

In [85]:
Test['Con_Time_of_day'] = ['Morning' if x< 43200 else 'Afternoon' for x in Test['Confirmation - Time']]

In [86]:
Test['Platform Type'] = ['Busy' if x == 3  else 'Not_busy' for x in Test['Platform Type']]

In [87]:
Test.drop(['Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Pickup - Time'], axis=1, inplace=True)

In [88]:
Test.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Rider Id,Pickup_to_Destination,Con_Time_of_day
0,Busy,Business,27,3,8,Rider_Id_192,Pumwani-Nairobi,Afternoon
1,Busy,Business,17,5,5,Rider_Id_868,Nairobi-Nairobi,Afternoon
2,Busy,Business,27,4,5,Rider_Id_26,Nairobi-Pumwani,Morning
3,Busy,Business,17,1,5,Rider_Id_685,Nairobi-Nairobi,Afternoon
4,Busy,Business,11,2,6,Rider_Id_858,Nairobi-Nairobi,Morning


## Data preproccessing

In order to carry out the regression model we have to first prepare all the inputs for that model. One requirement of the regression models is that all the inputs must be numerical. Therefore all of the categorical data must be converted into numerical data, we do this by utilizing the onehot encoding in the pd.get_dummies() method. In order  for there to be consistency in the variables the the Test data and the Train data need to be combined but at the same time kept separate, we do this by introducing a "train" column for both datasets so that we are able  to distinguish the 2 datasets.

In [89]:
Train['train'] = 1
Test['train'] = 0

In [90]:
combined = pd.concat([Train,Test])

In [91]:
combined = combined.merge(Riders, how='left', on = 'Rider Id')

In [92]:
combined

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Distance (KM),Rider Id,Time from Pickup to Arrival,Pickup_to_Destination,Con_Time_of_day,train,Average_Rating
0,Busy,Business,9,5,4,Rider_Id_432,745.0,Nairobi-Nairobi,Morning,1,13.8
1,Busy,Personal,12,5,16,Rider_Id_856,1993.0,Pumwani-Nairobi,Morning,1,13.6
2,Busy,Business,30,2,3,Rider_Id_155,455.0,Pumwani-Nairobi,Afternoon,1,12.5
3,Busy,Business,15,5,9,Rider_Id_855,1341.0,Nairobi-Nairobi,Morning,1,14.5
4,Not_busy,Personal,13,1,9,Rider_Id_770,1214.0,Nairobi-Nairobi,Morning,1,14.1
...,...,...,...,...,...,...,...,...,...,...,...
28264,Busy,Business,7,1,5,Rider_Id_362,NaN,Nairobi-Nairobi,Morning,0,13.1
28265,Busy,Business,10,3,22,Rider_Id_669,NaN,Nairobi-Pumwani,Afternoon,0,13.9
28266,Busy,Business,5,3,10,Rider_Id_379,NaN,Nairobi-Nairobi,Afternoon,0,13.7
28267,Busy,Personal,29,2,18,Rider_Id_474,NaN,Kikuyu-Nairobi,Afternoon,0,14.1


In [93]:
combined = combined[['Platform Type', 'Distance (KM)', 'Rider Id','Personal or Business','Pickup_to_Destination',
                     'Con_Time_of_day','Placement - Day of Month',
                     'Placement - Weekday (Mo = 1)','Average_Rating','train','Time from Pickup to Arrival']]

In [94]:
combined.columns = [col.replace(' ','_') for col in combined.columns]

combined.columns = [col.replace('(','') for col in combined.columns]
combined.columns = [col.replace(')','') for col in combined.columns]


In [95]:
combined

,Platform_Type,Distance_KM,Rider_Id,Personal_or_Business,Pickup_to_Destination,Con_Time_of_day,Placement_-_Day_of_Month,Placement_-_Weekday_Mo_=_1,Average_Rating,train,Time_from_Pickup_to_Arrival
0,Busy,4,Rider_Id_432,Business,Nairobi-Nairobi,Morning,9,5,13.8,1,745.0
1,Busy,16,Rider_Id_856,Personal,Pumwani-Nairobi,Morning,12,5,13.6,1,1993.0
2,Busy,3,Rider_Id_155,Business,Pumwani-Nairobi,Afternoon,30,2,12.5,1,455.0
3,Busy,9,Rider_Id_855,Business,Nairobi-Nairobi,Morning,15,5,14.5,1,1341.0
4,Not_busy,9,Rider_Id_770,Personal,Nairobi-Nairobi,Morning,13,1,14.1,1,1214.0
...,...,...,...,...,...,...,...,...,...,...,...
28264,Busy,5,Rider_Id_362,Business,Nairobi-Nairobi,Morning,7,1,13.1,0,NaN
28265,Busy,22,Rider_Id_669,Business,Nairobi-Pumwani,Afternoon,10,3,13.9,0,NaN
28266,Busy,10,Rider_Id_379,Business,Nairobi-Nairobi,Afternoon,5,3,13.7,0,NaN
28267,Busy,18,Rider_Id_474,Personal,Kikuyu-Nairobi,Afternoon,29,2,14.1,0,NaN


#### Here we perform the one hot encoding

In [96]:
df_dummies = pd.get_dummies(combined,columns=['Platform_Type','Rider_Id','Pickup_to_Destination',
                                              'Con_Time_of_day','Placement_-_Day_of_Month','Placement_-_Weekday_Mo_=_1','Personal_or_Business'], drop_first=True)

In [97]:
df_dummies.head()

,Distance_KM,Average_Rating,train,Time_from_Pickup_to_Arrival,Platform_Type_Not_busy,Rider_Id_Rider_Id_1,Rider_Id_Rider_Id_10,Rider_Id_Rider_Id_100,Rider_Id_Rider_Id_101,Rider_Id_Rider_Id_102,...,Placement_-_Day_of_Month_29,Placement_-_Day_of_Month_30,Placement_-_Day_of_Month_31,Placement_-_Weekday_Mo_=_1_2,Placement_-_Weekday_Mo_=_1_3,Placement_-_Weekday_Mo_=_1_4,Placement_-_Weekday_Mo_=_1_5,Placement_-_Weekday_Mo_=_1_6,Placement_-_Weekday_Mo_=_1_7,Personal_or_Business_Personal
0,4,13.8,1,745.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,16,13.6,1,1993.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,3,12.5,1,455.0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
3,9,14.5,1,1341.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,9,14.1,1,1214.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


We then separate the the test and the train dataset. Now the train and the test datasets are similar.

In [98]:
Test2 = df_dummies[df_dummies['train']== 0]
Test2.drop(['train'],axis=1,inplace=True)

Train2 = df_dummies[df_dummies['train']==1]
Train2.drop(['train'],axis=1, inplace=True)

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Delete the 'Time_from_Pickup_to_Arrival' column from the test dataset

In [99]:
Test2.drop(['Time_from_Pickup_to_Arrival'], axis=1,inplace=True)

In [100]:
column_titles = [col for col in Train2.columns if col !=
                 'Time_from_Pickup_to_Arrival'] + ['Time_from_Pickup_to_Arrival']
Train3 = Train2.reindex(columns=column_titles)

## Model implementation

In [101]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [102]:
x = Train3.iloc[:,:-1]
y = Train3.iloc[:,-1]

#### Feature Scaling

Feature scaling has been implemented on the columns Distance_KM and the average rating column in order to have the values on the same scale.

In [103]:
scaler = StandardScaler()
x.iloc[:,:2] = scaler.fit_transform(x.iloc[:,:2])

In [104]:
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    shuffle=False)

In [105]:
reg = BayesianRidge()
reg.fit(X_train, y_train)

BayesianRidge()

In [106]:
train_boost = reg.predict(X_train)
test_boost = reg.predict(X_test)

## Model evaluation 

The Root Mean squared error is a measure of quality of a linear regression, it is the sum the squared sum of the difference between actual  result and the estimated result. Therefore this implies that the smaller the RMSE error the better the model is at predicting.

The $R^{2}$ value on the other hand is a measure of the strength of the rlationship between the independent and dependent variables, the $R^{2}$ value is normaly between 0 and 1.  0 meaning that the relationship is non existant or its very week 1 meaning that it is very strong. From the previous statement one can conclude that the the higher the R_squared value the better the model but that is not necessarily true.

Models with Low $R^{2}$ values aren't necessarily bad because in some cases where human behavior is being predicted the $R^{2}$ value is below 50% humans are quiet unpredictable. In our case we have a low $R^{2}$ value but variables like distance are statistically significant (as proven in the EDA) which enables us to draw insight on how the change in distance influences the the dependent variable.

The simple $R^{2}$ is not enough it can prove to be miss leading in its nature because for multiple linear regression as you add more independent variables the $R^{2}$ value increases it never decreases so therefore to overcome this one needs to calculate the adjusted $R^{2}$ which icorparates the amount of variables in the model and adds a penalty for each variable added that is of no significance to the dependent variable.

In [107]:
from math import sqrt
from  sklearn import metrics
print('Ridge RSME :', sqrt(metrics.mean_squared_error(y_test, test_boost)))
r = metrics.r2_score(y_train, train_boost)
print('r_squared :', r)

Ridge RSME : 745.0781656189132
r_squared : 0.44167767066124564


In [108]:
p = len(list(X_train.columns))
n = len(X_train)
R_squred_adjusted = 1 - (1- r)*((n-1)/(n-p-1))
print ('adjusted R_squared is :', R_squred_adjusted)

adjusted R_squared is : 0.40557546718212467


## Predicting Test data

Because the train data had been scaled, in order to get accurate results the Test data also needs to be transformed using the same scale as the the train data

In [109]:
Test2.iloc[:,:2] = scaler.transform(Test2.iloc[:,:2])

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [110]:
Testr = reg.predict(Test2)
print(Testr)

[1624.51654164 1564.34350035 1080.03958398 ... 1494.62883665 2393.61462658
 1681.61349433]


## Creating Submission

In [111]:
Test_res = pd.DataFrame(Testr, columns=['Time from Pickup to Arrival'])
Tests = pd.read_csv('Test.csv')
Tests['Time from Pickup to Arrival'] = Test_res

In [112]:
Submission2 = Tests[['Order No','Time from Pickup to Arrival']].copy()

In [113]:
Submission2.to_csv('Submission5.csv',index=False)

In [115]:
import pickle

model_save_path = 'mlr_Ridge_model.pkl'

with open(model_save_path, 'wb') as file:
    pickle.dump(reg, file)